# MLOps End to End Workflow (II)

Implementation of an end-to-end ML Ops workflow for the use case to detect fraudulent credit card transactions, see [Kaggle dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

This set of notebooks cover:

[Experimentation](01-experimentation.ipynb):
1. Set up: Creation of the Vertex Dataset, extraction of the schema
2. Implementation of a TFX pipeline

[CICD](02-cicd.ipynb):

3. Deployment of the Vertex AI Pipeline through a CI/CD process
4. Deployment of a Continuous Training pipeline that can be triggered via Pub/Sub and produces a model in the Model Registry
5. Deployment of the Inference Pipeline consisting of a Cloud Function that retrieves features from Feature Store and calls the model on a Vertex AI Endpoint
6. Deployment of the model to a Vertex AI Endpoint through a CI/CD process.

[Prediction](03-prediction.ipynb):

7. Deploy the model to an endpoint
8. Create a test prediction


### Configuration

In [ ]:
#%load_ext autoreload
#%autoreload 2

import os
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from google.cloud import bigquery
import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai

import yaml
import os

In [ ]:
with open('mainconfig.yaml') as f:
    main_config = yaml.safe_load(f)

# select your config    
main_config = main_config['creditcards']

In [ ]:
PROJECT = main_config['project'] 
REGION = main_config['region'] 
DOCKER_REPO = main_config['docker_repo']

SERVICE_ACCOUNT = main_config['service_account']

# BigQuery and data locations

BQ_SOURCE_TABLE= main_config['bq']['source_table'] # raw input
ML_TABLE = main_config['bq']['ml_table'] # the one we will use for the training

BQ_DATASET_NAME = main_config['bq']['dataset']
BQ_LOCATION = main_config['bq']['location'] # multiregion provides more resilience

VERTEX_DATASET_NAME = main_config['vertex_dataset_name']

RAW_SCHEMA_DIR = main_config['raw_schema_dir']

BUCKET =  main_config['bucket']

# TFX and model config

# model version
VERSION = main_config['version']


MODEL_DISPLAY_NAME = f'{VERTEX_DATASET_NAME}-classifier-{VERSION}'
WORKSPACE = f'gs://{BUCKET}/{VERTEX_DATASET_NAME}'

MLMD_SQLLITE = 'mlmd.sqllite'
ARTIFACT_STORE = os.path.join(WORKSPACE, 'tfx_artifacts_interactive')
MODEL_REGISTRY = os.path.join(WORKSPACE, 'model_registry')
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)

ENDPOINT_DISPLAY_NAME = f'{VERTEX_DATASET_NAME}-classifier'

FEATURESTORE_ID = main_config['featurestore_id']

CF_REGION = main_config['cloudfunction_region']

DATAFLOW_SUBNETWORK = f"https://www.googleapis.com/compute/v1/projects/{PROJECT}/regions/{REGION}/subnetworks/{main_config['dataflow']['subnet']}"
DATAFLOW_SERVICE_ACCOUNT = main_config['dataflow']['service_account']

CLOUDBUILD_SA = f'projects/{PROJECT}/serviceAccounts/{SERVICE_ACCOUNT}'

LIMIT=main_config['limit']

In [ ]:
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Service Account:", SERVICE_ACCOUNT)

vertex_ai.init(
    project=PROJECT,
    location=REGION
)

## Unit Testing

In [ ]:
os.environ["VERTEX_DATASET_NAME"] = VERTEX_DATASET_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}"
os.environ["MODEL_REGISTRY_URI"] = os.path.join(os.environ["GCS_LOCATION"], "model_registry")
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["DATAFLOW_IMAGE_URI"] = f"{DOCKER_REPO}/dataflow:latest"
os.environ["TFX_IMAGE_URI"] = f"{DOCKER_REPO}/vertex:latest"
os.environ["ENABLE_CACHE"] = "1"
os.environ["SUBNETWORK"] = DATAFLOW_SUBNETWORK
os.environ["SERVICE_ACCOUNT"] = DATAFLOW_SERVICE_ACCOUNT
os.environ["BQ_LOCATION"] = BQ_LOCATION
os.environ["BQ_DATASET_NAME"] = BQ_DATASET_NAME
os.environ["ML_TABLE"] = ML_TABLE
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/e2e_tests"
os.environ["SUBNETWORK"] = DATAFLOW_SUBNETWORK

In [ ]:
os.environ["UPLOAD_MODEL"] = "0"
os.environ["ACCURACY_THRESHOLD"] = "-0.1"    # NB Negative accuracy threshold makes no sense - allows everything
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [ ]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

In [ ]:
!python -m pytest src/tests/datasource_utils_tests.py -s

In [ ]:
!python -m pytest src/tests/model_tests.py -s

#### End to end pipeline unit test

In [ ]:
!python -m pytest src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

## Deploy to Vertex AI Pipelines

In [ ]:
config.BEAM_RUNNER

In [ ]:
from src.tfx_pipelines import config
import importlib

importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Create Repo for Images

In [ ]:
# Repo should has been created in the Terraform automation stage
#! gcloud artifacts repositories create {VERTEX_DATASET_NAME} --location={REGION} --repository-format=docker

### Build Dataflow Worker Image

In [ ]:
# You can also use build/Dockerfile.dataflow in case Internet access is not allowed
!cp build/Dockerfile.dataflow_internet Dockerfile

In [ ]:
os.environ["DOCKER_REPO"] = f"{DOCKER_REPO}/vertex:latest"
!gcloud builds submit --project=$PROJECT --billing-project=$PROJECT --region $REGION --tag $DOCKER_REPO/dataflow:latest . --timeout=15m --machine-type=e2-highcpu-8 --suppress-logs

### Build Vertex worker image

In [ ]:
!echo $TFX_IMAGE_URI

In [ ]:
!echo $PYTHONPATH

In [ ]:
!cp build/Dockerfile.vertex_internet Dockerfile
!gcloud builds submit --project=$PROJECT --billing-project=$PROJECT --region $REGION --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8 --suppress-logs

In [ ]:
PIPELINES_STORE = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/compiled_pipelines/"
!gsutil cp {pipeline_definition_file} {PIPELINES_STORE}
PIPELINES_STORE

### Compile the pipeline

In [ ]:
from src.tfx_pipelines import config, runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_training_pipeline(pipeline_definition_file)

### Submit Vertex AI Pipelines run

In [ ]:
from google.cloud.aiplatform import pipeline_jobs
    
job = pipeline_jobs.PipelineJob(template_path = pipeline_definition_file,
                                display_name=VERTEX_DATASET_NAME,
                                #enable_caching=False,
                                parameter_values={
                                    'learning_rate': 0.003,
                                    'batch_size': 512,
                                    'steps_per_epoch': int(config.TRAIN_LIMIT) // 512,
                                    'hidden_units': '128,128',
                                    'num_epochs': 30,
                                })

job.run(sync=False, service_account=DATAFLOW_SERVICE_ACCOUNT)

## Deploy Continuous Training Pipeline ("CI/CD")

* Deploy Pub/Sub topic to listen to retraining triggers
* Deploy Cloud Function to listen to Pub/Sub topic and trigger Vertex AI Pipeline
* Deploy Vertex AI Pipeline

### Build CI/CD image

In [ ]:
CICD_IMAGE_URI = f"{DOCKER_REPO}/cicd:latest"
CICD_IMAGE_URI

In [ ]:
# For the CICD container, we are just adding the build/* dir
!cp build/Dockerfile.cicd_internet build/Dockerfile
!gcloud builds submit --project=$PROJECT --billing-project=$PROJECT --region $REGION --tag $CICD_IMAGE_URI build/. --timeout=15m --machine-type=e2-highcpu-8 --suppress-logs

### Automate the deployment of the Training Pipeline using Cloud Build
***Important*** you should commit the code to the git repo since the next build process will checkout the code from the repo.

In [ ]:
REPO_URL = main_config['git']['repo_url']
BRANCH = main_config['git']['branch']


GCS_LOCATION = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/"
TEST_GCS_LOCATION = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/e2e_tests"
CI_TRAIN_LIMIT = 1000
CI_TEST_LIMIT = 100
CI_UPLOAD_MODEL = 0
CI_ACCURACY_THRESHOLD = -0.1 # again setting accuracy threshold to negative
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'latest'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINES_STORE = f"{GCS_LOCATION}compiled_pipelines/"

TFX_IMAGE_URI = f"{DOCKER_REPO}/vertex:{VERSION}"
DATAFLOW_IMAGE_URI = f"{DOCKER_REPO}/dataflow:latest"

REPO_NAME = REPO_URL.split('/')[-1]
DESCR=f'"Deploy train pipeline to GCS from {BRANCH}"'


SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_BQ_LOCATION={BQ_LOCATION},\
_BQ_DATASET_NAME={BQ_DATASET_NAME},\
_ML_TABLE={ML_TABLE},\
_VERTEX_DATASET_NAME={VERTEX_DATASET_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_CI_TRAIN_LIMIT={CI_TRAIN_LIMIT},\
_CI_TEST_LIMIT={CI_TEST_LIMIT},\
_CI_UPLOAD_MODEL={CI_UPLOAD_MODEL},\
_CI_ACCURACY_THRESHOLD={CI_ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_DATAFLOW_IMAGE_URI={DATAFLOW_IMAGE_URI},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE},\
_SUBNETWORK={DATAFLOW_SUBNETWORK},\
_GCS_BUCKET={BUCKET}/cloudbuild,\
_SERVICE_ACCOUNT={DATAFLOW_SERVICE_ACCOUNT},\
_WORKDIR={REPO_NAME}\
"""
!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit build/known_hosts.github.zip --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --project=$PROJECT --billing-project=$PROJECT --region $REGION --suppress-logs

### (Optional for Cloud Sources Repositories) Define the trigger that will deploy the pipeline after a commit

In [ ]:
!echo gcloud beta builds triggers create cloud-source-repositories --repo={REPO_NAME} --branch-pattern=^{BRANCH}$ --description={DESCR} --build-config=mlops-creditcard/build/pipeline-deployment.yaml --substitutions={SUBSTITUTIONS} --billing-project={PROJECT}  --service-account={TRIGGER_SA}

### Set up the trigger for the Training Pipeline

* Deploy Pub/Sub Topic
* Deploy Cloud Function that listens to the topic and triggers the pipeline

In [ ]:
PUBSUB_TOPIC = f'trigger-{PIPELINE_NAME}'
CLOUD_FUNCTION_NAME = f'trigger-{PIPELINE_NAME}-fn'
GCS_PIPELINE_FILE_LOCATION = os.path.join(PIPELINES_STORE, f'{PIPELINE_NAME}.json')

#### Create Pub/Sub Topic

In [ ]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

#### Deploy Cloud Function

In [ ]:
ENV_VARS=f"""\
PROJECT={PROJECT},\
REGION={REGION},\
GCS_PIPELINE_FILE_LOCATION={GCS_PIPELINE_FILE_LOCATION},\
SERVICE_ACCOUNT={SERVICE_ACCOUNT},\
PIPELINE_NAME={PIPELINE_NAME}
"""

!echo {ENV_VARS}

In [ ]:
!rm -rf src/pipeline_triggering/.ipynb_checkpoints

In [ ]:
!gcloud functions deploy {CLOUD_FUNCTION_NAME} --gen2 \
    --region={CF_REGION} \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python38 \
    --source=src/pipeline_triggering\
    --entry-point=trigger_pipeline\
    --stage-bucket={BUCKET}\
    --ingress-settings=internal-only\
    --service-account={SERVICE_ACCOUNT}\
    --update-env-vars={ENV_VARS} 

#### Test triggering the pipeline with a Pub/Sub message

In [ ]:
from google.cloud import pubsub
import json

publish_client = pubsub.PublisherClient()
topic = f'projects/{PROJECT}/topics/{PUBSUB_TOPIC}'
data = {
    'num_epochs': 7,
    'learning_rate': 0.0015,
    'batch_size': 512,
    'steps_per_epoch': int(config.TRAIN_LIMIT) // 512,
    'hidden_units': '256,126'
}
message = json.dumps(data)

_ = publish_client.publish(topic, message.encode())

Check the console to see that it's running.

We now have:

* A pipeline that can be run to test and deploy new training pipelines
* A triggering mechanism to programmatically trigger new training runs
* A training run finish with a new model in the Vertex AI Model Registry


### Preparation

#### Vertex AI Endpoint

In [ ]:
ENDPOINT_DISPLAY_NAME

In [ ]:
from build.utils import create_endpoint

endpoint = create_endpoint(PROJECT, REGION, ENDPOINT_DISPLAY_NAME)
endpoint

In [ ]:
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient

In [ ]:
from feature_store import feature_store as fs

Create Feature Store

In [ ]:
fs.create_fs(PROJECT, REGION, FEATURESTORE_ID, "Feature Store for credit card use case")

In [ ]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient, FeatureSelector
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import feature as feature_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import ListFeaturestoresRequest, CreateFeaturestoreRequest, Featurestore, ListEntityTypesRequest

from google.protobuf.timestamp_pb2 import Timestamp
from google.cloud.aiplatform_v1beta1.types import featurestore_monitoring as featurestore_monitoring_pb2
from google.protobuf.duration_pb2 import Duration


API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"  
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
parent = f'{admin_client.common_location_path(PROJECT, REGION)}/featurestores/{FEATURESTORE_ID}'
request = ListEntityTypesRequest(parent=parent)

# Make the request
page_result = admin_client.list_entity_types(request=request)

# Handle the response
[x.name.split('/')[-1] for x in page_result]


In [ ]:
admin_client.featurestore_path(PROJECT, REGION, FEATURESTORE_ID)

#### Create an entity with features, generate some data and upload it

In [ ]:
entity = 'user'
entity_descr = 'User ID'
features = ['v27', 'v28']

In [ ]:
fs.create_entity(PROJECT, REGION, FEATURESTORE_ID, entity, entity_descr, features)

In [ ]:
import random

filename = f'features_{entity}.csv'

with open(filename, 'w') as f:
    line = f'{entity},{",".join(features)}\n'
    f.write(line)
    for i in range(100):
        f.write(f'user{i},{random.random()},{random.random()}\n')

In [ ]:
!tail -20 {filename}

In [ ]:
BUCKET

In [ ]:
!gsutil cp {filename} gs://{BUCKET}/{filename} 

In [ ]:
gcs_uris = [f'gs://{BUCKET}/{filename}']

fs.ingest_entities_csv(PROJECT, REGION, FEATURESTORE_ID, entity, features, gcs_uris)

Test reading some features back

In [ ]:
features_data = {}
for i in range(90,102):
    entity_id = f'user{i}'
    features_data[entity_id] = fs.read_features(PROJECT, REGION, FEATURESTORE_ID, entity, features, entity_id)

features_data

In [ ]:
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

In [ ]:
!python -m pytest src/tests/model_deployment_tests.py::test_model_artifact -s

#### Deploy Model to Endpoint

In [ ]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

#### Test model on Endpoint

In [ ]:
!python -m pytest src/tests/model_deployment_tests.py::test_model_endpoint

#### Run the pipeline

In [ ]:
REPO_URL = main_config['git']['repo_url']
BRANCH = main_config['git']['branch']

f'{REPO_URL}:{BRANCH}'

In [ ]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
_GCS_BUCKET={BUCKET}/cloudbuild,\
_SERVICE_ACCOUNT={SERVICE_ACCOUNT},\
_WORKDIR={REPO_NAME}\
"""

SUBSTITUTIONS

### Test the build and define a manual trigger

In [ ]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --billing-project {PROJECT} --suppress-logs --async

In [ ]:
DESCR=f'"Deploy model from branch {BRANCH}"'

In [ ]:
!gcloud alpha builds triggers create manual --repo={REPO_URL} --repo-type=CLOUD_SOURCE_REPOSITORIES --branch={BRANCH} --description={DESCR} --build-config=mlops-creditcard/build/model-deployment.yaml --substitutions={SUBSTITUTIONS} --billing-project={PROJECT} --service-account={CLOUDBUILD_SA}

In [ ]:
endpoints = vertex_ai.Endpoint.list(
    filter=f'display_name={ENDPOINT_DISPLAY_NAME}',
    order_by="update_time"
)

if len(endpoints) == 0:
    print(f'No endpoints found with name {ENDPOINT_DISPLAY_NAME}')
endpoint = endpoints[-1]

os.environ['ENDPOINT_NAME'] = endpoint.name

entity = 'user'
os.environ['ENTITY'] = entity
os.environ['FEATURESTORE_ID'] = FEATURESTORE_ID

PREDICT_CLOUD_FUNCTION_NAME = "predict-" + PIPELINE_NAME + "-fn"
PREDICT_CLOUD_FUNCTION_NAME

In [ ]:
from src.tests.model_deployment_tests import test_instance

import base64

if 'V27' in test_instance:
    del test_instance['V27']
if 'V28' in test_instance:
    del test_instance['V28']
test_instance['userid'] = 'user99'

test_instance

In [ ]:
from flask import Flask

In [ ]:
from src.prediction_cf.main import predict

app = Flask('test')
ctx = app.test_request_context(json=test_instance)
request = ctx.request

pred_retval = predict(request)
pred_retval

In [ ]:
GOOGLE_FUNCTION_SOURCE ='src/prediction_cf/main.py'

ENV_VARS=f"""\
PROJECT={PROJECT},\
REGION={REGION},\
ENDPOINT_NAME={endpoint.name},\
ENTITY={entity},\
FEATURESTORE_ID={FEATURESTORE_ID}
"""

!echo {ENV_VARS}

In [ ]:
!rm -rf src/prediction_cf/.ipynb_checkpoints

In [ ]:
!gcloud functions deploy {PREDICT_CLOUD_FUNCTION_NAME} \
    --set-build-env-vars=GOOGLE_FUNCTION_SOURCE={GOOGLE_FUNCTION_SOURCE} \
    --region={CF_REGION} \
    --runtime=python37 \
    --trigger-http \
    --source=. \
    --entry-point=predict\
    --stage-bucket={BUCKET}\
    --ingress-settings=internal-only\
    --service-account={SERVICE_ACCOUNT}\
    --set-env-vars={ENV_VARS} \
     --billing-project $PROJECT

#### Test the prediction cloud function

You can test it using a `curl` command, but this has to be executed from the same VPC that the Cloud Function is deployed in:

```
curl -m 70 -X POST https://PROJECT.cloudfunctions.net/predict-creditcards-classifier-v01-train-pipeline-fn \
-H "Authorization:bearer $(gcloud auth print-identity-token)" \
-H "Content-Type:application/json" \
-d '{"V1": [-0.906611], "V2": [-0.906611], "V3": [-0.906611], "V4": [-0.906611], "V5": [-0.906611], "V6": [-0.906611], "V7": [-0.906611], "V8": [-0.906611], "V9": [-0.906611], "V10": [-0.906611], "V11": [-0.906611], "V12": [-0.906611], "V13": [-0.906611], "V14": [-0.906611], "V15": [-0.906611], "V16": [-0.906611], "V17": [-0.906611], "V18": [-0.906611], "V19": [-0.906611], "V20": [-0.906611], "V21": [-0.906611], "V22": [-0.906611], "V23": [-0.906611], "V24": [-0.906611], "V25": [-0.906611], "V26": [-0.906611], "Amount": [15.99], "userid": "user99"}'
```

You deploy a VM in the same VPC and run the command from there.

Perhaps an easier way to test is test it from the Testing tab on the [web Console](https://console.cloud.google.com/functions/list).

In [ ]:
import json

json.dumps(test_instance)